## Experiment 1: (2022-07-28~)
- - -
#### 0. Preprocessing / (Done on seperate file)
Urbansound8k Dataset i){8732(<=4s).wav files/10 folders(must be separated)} => Noise data    
MTG-jamendo Dataset i){}=> Music data   
For easy labeling, we need to save the labels along with the data ( X.npy = [ data , label ] )

#### 1. Initialize input pipeline


#### 2. Model initialization


#### 3. Training & Evaluation

- - -

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_addons as tfa

import pathlib

import librosa

import soundfile as sf

from tqdm import tqdm

from IPython.display import clear_output

autotune = tf.data.AUTOTUNE

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            tf.config.experimental.set_virtual_device_configuration(gpu, [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*11)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


## 1. Initializing Input Pipeline   
(planned 22.08.08)   
* First define the raw tf.data.Datasets.   
* Make a function that preprocesses the training datasets.
* Use tf.data.Dataset.map to create the fully processed dataset.
* Then make a function that finds the labels for each of the training datasets.   
* Use tf.data.Dataset.map to create the label Dataset.   
* Then combine them with tf.data.Dataset.zip.   
* Then Shuffle & Configure for performance

In [41]:
IMAGE_SIZE = 513*345
IMAGE_DIM = [513,345]

# Training
batch_size = 1

In [2]:
base_path = pathlib.Path('D:/전재원졸업연구/music-discern/Datasets/') # Edit on other computers

#Urbansound is nested in Datasets/Urbansound8K/audio/fold**
#Urbansound is already separated into batches, which need to be preserved.
urban_metadata_path = pathlib.Path(base_path/'Urbansound8K/metadata')
urban_audio_folders_path = pathlib.Path(base_path/'Urbansound8K/processed')
urban_audio_path_list = [x for x in urban_audio_folders_path.iterdir()]
temp = urban_audio_path_list.pop(1)
urban_audio_path_list.append(temp)

#Jamendo is nested in Datasets/MTGJamendo/audio
#Jamendo is not separated in batches, so we need to separate it into 10 batches, matching urbansound.
jamendo_metadata_path = pathlib.Path(base_path/'MTGJamendo/metadata')
jamendo_audio_folders_path = pathlib.Path(base_path/'MTGJamendo/processed')
jamendo_audio_path_list = [x for x in jamendo_audio_folders_path.iterdir()]
temp = jamendo_audio_path_list.pop(1)
jamendo_audio_path_list.append(temp)


#for later -> add a path for any .wav file that I want to observe.
test_subjects_path = pathlib.Path("./")

Train = [0,1,2,3,4,5,6,7,8]
Test = [9] #Seperation, for later evaluation in Training step B

print(urban_audio_path_list)
print(jamendo_audio_path_list)

[WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold1'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold2'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold3'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold4'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold5'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold6'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold7'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold8'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold9'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/Urbansound8K/processed/fold10')]
[WindowsPath('D:/전재원졸업연구/music-discern/Datasets/MTGJamendo/processed/fold1'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/MTGJamendo/processed/fold2'), WindowsPath('D:/전재원졸업연구/music-discern/Datasets/MTGJam

In [40]:
dataset_for_training_A = "empty"
dataset_for_training_B_train = "empty"
dataset_for_training_B_test = "empty"

dataset_for_training_A = tf.data.Dataset.list_files(str(urban_audio_folders_path/'*/*.npy'))
dataset_for_training_A = dataset_for_training_A.concatenate( tf.data.Dataset.list_files(str(jamendo_audio_folders_path/'*/*.npy')) )

for k in Train:
    temp_path_urban = str(urban_audio_path_list[k])
    temp_path_jam = str(jamendo_audio_path_list[k])
    if dataset_for_training_B_train == "empty":
        dataset_for_training_B_train = tf.data.Dataset.list_files(temp_path_urban + '/*.npy')
        dataset_for_training_B_train = dataset_for_training_B_train.concatenate( tf.data.Dataset.list_files(temp_path_jam + '/*.npy') )
    else:
        dataset_for_training_B_train = dataset_for_training_B_train.concatenate( tf.data.Dataset.list_files(temp_path_urban + '/*.npy') )
        dataset_for_training_B_train = dataset_for_training_B_train.concatenate( tf.data.Dataset.list_files(temp_path_jam + '/*.npy') )

for k in Test:
    temp_path_urban = str(urban_audio_path_list[k])
    temp_path_jam = str(jamendo_audio_path_list[k])
    if dataset_for_training_B_test == "empty":
        dataset_for_training_B_test = tf.data.Dataset.list_files(temp_path_urban + '/*.npy')
        dataset_for_training_B_test = dataset_for_training_B_test.concatenate( tf.data.Dataset.list_files(temp_path_jam + '/*.npy') )
    else:
        dataset_for_training_B_test = dataset_for_training_B_test.concatenate( tf.data.Dataset.list_files(temp_path_urban + '/*.npy') )
        dataset_for_training_B_test = dataset_for_training_B_test.concatenate( tf.data.Dataset.list_files(temp_path_jam + '/*.npy') )

# returns dataset for training A, dataset for training B Train, dataset for training B Test

In [42]:
for f in dataset_for_training_A.take(1):
    print(f)
    print(tf.strings.split( tf.strings.split( f,os.sep )[-1],sep="_" )[1])
    A = np.load(f.numpy(),allow_pickle=True)
    print(A.dtype)
    img = tf.io.read_file(f)
    img = tf.io.decode_raw(img,tf.complex64)
    img = tf.slice(img,begin=[16],size=[IMAGE_SIZE])
    img = tf.reshape(img,[513,345])
    print(img)

tf.Tensor(b'D:\\\xec\xa0\x84\xec\x9e\xac\xec\x9b\x90\xec\xa1\xb8\xec\x97\x85\xec\x97\xb0\xea\xb5\xac\\music-discern\\Datasets\\Urbansound8K\\processed\\fold8\\71177-8-2-0_noise_0.npy', shape=(), dtype=string)
tf.Tensor(b'noise', shape=(), dtype=string)
complex64
tf.Tensor(
[[ 9.2683859e-02+0.0000000e+00j -2.2940416e-02+2.1145932e-01j
  -1.7487489e-01-3.4585261e-01j ...  1.1186663e-02-9.8369382e-03j
  -1.0906712e-02+1.3434681e-02j  8.1001036e-03-1.4528953e-02j]
 [-6.7503089e-03+1.3567344e-02j  6.4922301e-03-1.2852893e-02j
  -6.1244876e-03+1.2959077e-02j ...  2.8476156e-02+3.9916214e-02j
  -3.1287868e-02-4.9197387e-02j  3.2657418e-02+2.7656902e-02j]
 [-9.2140920e-03-1.9977847e-02j -1.6075812e-02+6.8027689e-03j
   1.4415859e-02+1.3888902e-02j ...  7.1546338e-02+9.6227443e-01j
  -6.8956472e-02-7.0292163e-01j  2.0028234e-01-1.3238974e-01j]
 ...
 [ 1.9950146e-06-2.6052321e-06j  1.4626867e-06+1.5176863e-06j
  -6.7456091e-07-4.7771567e-07j ...  2.9791694e-04+2.5267011e-04j
  -9.1383525e-04-3.3

In [46]:
def process_image(image):
    #make the images into spectrograms
    processed_img = tf.pow(tf.abs(image),2)
    #normalize the images?
    return processed_img

def process_path_A(path):
    #for training A, the label is just itself so we need to take that into account
    img = tf.io.read_file(path)
    img = tf.io.decode_raw(img,tf.complex64)
    img = tf.slice(img,begin=[1024//64],size=[IMAGE_SIZE]) # begin <- 1024//complex64
    img = tf.reshape(img,IMAGE_DIM)
    img = process_image(img)

    label = img
    return img, label

def process_path_B(path): #map_func
    #extract the 2d array and the label
    img = tf.io.read_file(path)
    img = tf.io.decode_raw(img,tf.complex64)
    img = tf.slice(img,begin=[1024//64],size=[IMAGE_SIZE]) # begin <- 1024//complex64
    img = tf.reshape(img,IMAGE_DIM)
    img = process_image(img)

    label = tf.strings.split( tf.strings.split( path, os.sep )[-1],sep="_" )[1]
    return img, label

In [47]:
dataset_A = dataset_for_training_A.map(process_path_A,num_parallel_calls=autotune)
dataset_B_train = dataset_for_training_B_train.map(process_path_B,num_parallel_calls=autotune)
dataset_B_test = dataset_for_training_B_test.map(process_path_B,num_parallel_calls=autotune)

In [48]:
next(iter(dataset_A))

(<tf.Tensor: shape=(513, 345), dtype=float32, numpy=
 array([[1.07342266e-01, 1.43307537e-01, 9.16463360e-02, ...,
         1.29235168e-05, 8.45194081e-05, 2.68683681e-04],
        [5.15958702e-04, 7.11368979e-04, 1.22292223e-03, ...,
         2.67169060e-04, 1.28484110e-03, 1.14161037e-02],
        [1.89742576e-02, 7.73821026e-03, 5.45850582e-03, ...,
         2.36976087e-01, 5.42790413e+00, 1.12960043e+01],
        ...,
        [2.15051355e-09, 2.34039588e-09, 2.26604979e-09, ...,
         1.06540756e-04, 4.63093347e-05, 4.82917065e-04],
        [1.16620632e-03, 8.16322630e-04, 1.00953941e-04, ...,
         5.04085002e-03, 1.10912994e-02, 1.03960363e-02],
        [1.79205090e-02, 1.78491306e-02, 1.14456872e-02, ...,
         5.85281100e-07, 5.70318434e-07, 5.05173830e-07]], dtype=float32)>,
 <tf.Tensor: shape=(513, 345), dtype=float32, numpy=
 array([[1.07342266e-01, 1.43307537e-01, 9.16463360e-02, ...,
         1.29235168e-05, 8.45194081e-05, 2.68683681e-04],
        [5.15958702e-04

In [ ]:
def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=autotune)
    return ds

dataset_A = configure_for_performance(dataset_A)
dataset_B_test = configure_for_performance(dataset_B_test)
dataset_B_train = configure_for_performance(dataset_B_train)

## 2. Model initialization

## 3. Training & Evaluation

![](/Experiment1abstract.jpg?raw=true)